In [1]:
import cv2
import joblib
import numpy as np
import pandas as pd 
from skimage.feature import greycomatrix, greycoprops

In [2]:
nb = joblib.load('KNN_ROI.pkl')

In [3]:
def preprocess_and_extract_features(image_path):
    #load image
    image = cv2.imread(image_path)

    height, width, channel = image.shape

    if height > width:
        crop_size = width
        y = int ((height-width)/2)
        x=0
    else:
        crop_size = height
        x = int ((width - height)/2)
        y=0

    cropped_image = image[y:y+crop_size, x:x+crop_size]

    gray = cv2.cvtColor(cropped_image,cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(
        gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU
    )
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2. CHAIN_APPROX_SIMPLE
    )
    cnt = max(contours, key=cv2.contourArea)

    x, y, w, h, =cv2.boundingRect(cnt)

    roi = cropped_image [y:y+h, x:x+w]

    resize_roi = cv2.resize(roi, (1080, 1080))

    # resize = cv2.resize(resize_roi, (512, 512))
    # cv2.imshow("resize", resize)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    #convert image to rgb
    rgb_image= cv2.cvtColor(resize_roi, cv2.COLOR_BGR2RGB)
    
    

    #rata rata rgb
    r_mean = np.mean(rgb_image[:,:,0])
    g_mean = np.mean(rgb_image[:,:,1])
    b_mean = np.mean(rgb_image[:,:,2])
    
    glcm_properties = ['correlation', 'homogeneity','dissimilarity','contrast',]

    image_gray = cv2.cvtColor(resize_roi, cv2.COLOR_BGR2GRAY)
    glcm = greycomatrix(image_gray, distances=[2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                levels=256, symmetric=True, normed=True)

    feature =[]
    glcm_props = [propery for name in glcm_properties for propery in greycoprops(glcm, name)[0]]
    
    for item in glcm_props:
        feature.append(item)

    #create feature array 
    x= np.array([r_mean,g_mean,b_mean] + feature[:])

    return x

In [4]:
def classify (image_path):
    #melakukan pre processing dan ekstrasi fitur
    x= preprocess_and_extract_features(image_path)

    #mengubah bentuk array
    x_reshaped = x.reshape(1,-1)

    #Membuat dataframe menggu nama fitur 
    glcm_properties = ['correlation', 'homogeneity','dissimilarity','contrast',]
    angles=[0,45,90,135]
    
    feature_names = ['R','G','B'] + [f'{prop} {angle}' for prop in glcm_properties for angle in angles]
    x_new = pd.DataFrame(data=x_reshaped, columns=feature_names)

    #predict label
    y_pred = nb.predict(x_new)

    #print label 
    print (f'Label: {y_pred[0]}')


In [12]:
#contoh 
# image_path= 'ROI/Testing/Tidak Layak/Data ke-3.jpg'
image_path='Support/DataBaru/Data (10).jpg'
classify(image_path)

Label: 2


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in v